# sequential

In [3]:
from pathlib import Path
from typing import Callable
import time

import requests


POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'http://fluentpython.com/data/flags'
DEST_DIR = Path('downloaded')

def save_flag(img: bytes, filename: str) -> None:
    (DEST_DIR / filename).write_bytes(img)

def get_flag(cc: str) -> bytes:
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    resp = requests.get(url)
    return resp.content

def download_many(cc_list: list[str]) -> int:
    for cc in sorted(cc_list):
        image = get_flag(cc)
        save_flag(image, f'{cc}.gif')
        print(cc, end=' ', flush=True)
    return len(cc_list)

def main(downloader: Callable[[list[str]], int]) -> None:
    t0 = time.perf_counter()
    count = downloader(POP20_CC)
    elapsed = time.perf_counter() - t0
    print(f'\n{count} downloads in {elapsed:.2f}s')

main(download_many)     

BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 downloads in 19.63s


# using concurrent futures

In [7]:
from concurrent import futures


def download_one(cc: str):
    image = get_flag(cc)
    save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc

def download_many(cc_list: list[str]) -> int:
    with futures.ThreadPoolExecutor() as executor:
        res = executor.map(download_one, sorted(cc_list))
    return len(list(res))

main(download_many)  

DE FR JP BD BR IN ID CD EG NG PK PH IR TR RU US CN ET VN MX 
20 downloads in 5.07s


same example but replaced the higher-level function map

In [31]:
def download_many(cc_list: list[str]) -> int:
    cc_list = cc_list[:5]
    with futures.ThreadPoolExecutor(max_workers=3) as executor:
        to_do: list[futures.Future] = []
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc)
            to_do.append(future)
            print(f'Scheduled for {cc}: {future}')
        futures_list = list(futures.as_completed(to_do))
        print('\n')
        for future in futures_list:
            res: str = future.result()
            print(f'{future} result: {res!r}')

    return len(futures_list)

main(download_many)

Scheduled for BR: <Future at 0x1f0f96d8ac0 state=running>
Scheduled for CN: <Future at 0x1f0f96cf6d0 state=running>
Scheduled for ID: <Future at 0x1f0f96cb7c0 state=running>
Scheduled for IN: <Future at 0x1f0f969a670 state=pending>
Scheduled for US: <Future at 0x1f0f969a7f0 state=pending>
BR CN ID US IN 

<Future at 0x1f0f96d8ac0 state=finished returned str> result: 'BR'
<Future at 0x1f0f96cf6d0 state=finished returned str> result: 'CN'
<Future at 0x1f0f96cb7c0 state=finished returned str> result: 'ID'
<Future at 0x1f0f969a7f0 state=finished returned str> result: 'US'
<Future at 0x1f0f969a670 state=finished returned str> result: 'IN'

5 downloads in 0.77s


ProcessPoolExecutor Example in 'ProcessPoolExecutor_Example.py'.